In [1]:
import urllib
import urllib.request
import urllib.parse
import bs4
import re
import os
import time
import pandas as pd
from concurrent.futures import ThreadPoolExecutor

In [ ]:
def deleteTag(x):
    return re.sub("<[^>]*>", "", x)

def getComments(code):
    def makeArgs(code, page):
        params = {
            'code': code,
            'type': 'after',
            'isActualPointWriteExecute': 'false',
            'isMileageSubscriptionAlready': 'false',
            'isMileageSubscriptionReject': 'false',
            'page': page
        }
        return urllib.parse.urlencode(params)

    def innerHTML(s, sl=0):
        ret = ''
        for i in s.contents[sl:]:
            if i is str:
                ret += i.strip()
            else:
                ret += str(i)
        return ret

    def fText(s):
        if len(s): return innerHTML(s[0]).strip()
        return ''

    retList = []
    colSet = set()
    print("Processing: %d" % code)
    page = 1

    # code데이터로 영화 제목 추출
    title = "https://movie.naver.com/movie/bi/mi/basic.nhn?code=" + str(code)
    html = urllib.request.urlopen(title).read()
    soup_title = bs4.BeautifulSoup(html, 'html.parser')
    name = soup_title.find('h3', {'class': 'h_movie'}).find('a').text

    while 1:
        try:
            f = urllib.request.urlopen(
                "http://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?" + makeArgs(code, page))
            data = f.read().decode('utf-8')
        except:
            break
        soup = bs4.BeautifulSoup(re.sub("&#(?![0-9])", "", data), "html.parser")
        cs = soup.select(".score_result li")
        
        if not len(cs): break
        for link in cs:
            try:
                url = link.select('.score_reple em a')[0].get('onclick')
            except:
                print(page)
                print(data)
                raise ""
            m = re.search('[0-9]+', url)
            if m:
                url = m.group(0)
            else:
                url = ''
            if url in colSet: return retList
            colSet.add(url)
            cat = fText(link.select('.star_score em'))
            cont = fText(link.select('.score_reple p'))
            cont = re.sub('<span [^>]+>.+?</span>', '', cont)
            retList.append((name, cont, cat))
        page += 1

    return retList

In [ ]:
def fetch(i):
    outname = 'comments/{}.csv'.format(i)
    data = []
    try:
        if os.stat(outname).st_size > 0: return
    except:
        None
    rs = getComments(i)
    # 해당 영화코드에 데이터가 없으면 스킵
    if not len(rs): return

    for idx, r in enumerate(rs):
        data.append([i, r[0],r[1],r[2].replace("'", "''").replace("\\", "\\\\")])

    df = pd.DataFrame(data)
    df.to_csv('comments/{}.csv'.format(i), encoding='utf=8')

    time.sleep(1)

In [ ]:
with ThreadPoolExecutor(max_workers=5) as executor:
    # 10000 ~ 200000 영화 고유코드로 전체 스캔
    for i in range(180000, 181000):
        executor.submit(fetch, i)